In [7]:
import os
from util import parse_kfold_results
from util import find_optimal_epoch
from util import analyze_convergence
from util import print_recommendations

In [8]:
spe = "yeast2"
models = ["DeepFE-PPI", "DeepTrio", "MyPPI", "PIPR", "TF-PPI"]
# model2epoch = {"DeepTrio" : 10}
model2epoch = {}
optimal_epochs = {}

for model in models:
    result_file = os.path.join("result", spe, f"{model}.txt")
    if not os.path.exists(result_file):
        continue
    
    with open(result_file, "r", encoding="utf-8") as fin:
        kfold_text = fin.read()

    df = parse_kfold_results(kfold_text)

    epoch_stats, optimal_epoch = find_optimal_epoch(df, model2epoch.get(model, None))
    if optimal_epoch is None:
        continue
    
    optimal_epochs[model] = optimal_epoch

    convergence_df = analyze_convergence(df)
    
    print("=" * 80)
    print(model)
    print("=" * 80)

    print_recommendations(df, epoch_stats, optimal_epoch, convergence_df)

    print("\n")

DeepFE-PPI

最优epoch: 16
--------------------------------------------------
综合评分: 0.9469

平均准确率: 94.54 (±0.65)
平均精确率: 96.81 (±0.48)
平均召回率: 92.13 (±0.92)
平均F1分数: 94.41 (±0.70)
平均ROC AUC: 94.54 (±0.66)
平均PR AUC: 93.13 (±0.86)

平均验证损失: 0.1724

收敛分析:
--------------------------------------------------
 fold  final_f1  stable_epoch  converged_early
    1    0.9441            11             True
    2    0.9358            11             True
    3    0.9489            11             True
    4    0.9439            11             True
    5    0.9397            11             True

前5个最佳epoch:
--------------------------------------------------
 epoch  composite_score  f1_mean  f1_std
    16          0.94695   0.9441  0.0070
    25          0.94694   0.9442  0.0048
    27          0.94673   0.9439  0.0068
    20          0.94650   0.9435  0.0047
    30          0.94611   0.9429  0.0048

建议:
--------------------------------------------------
建议选择epoch 16 - 模型收敛较早，训练效率高
模型表现稳定，各折间方差小


DeepTrio

最

In [9]:
print(f"Model\tAccuracy\tPrecision\tRecall\tF1\tROC AUC\tPR AUC\tComposite Score")
for model, optimal_epoch in optimal_epochs.items():
    accuracy = f"{optimal_epoch['accuracy_mean']*100:.2f}±{optimal_epoch['accuracy_std']*100:.2f}"
    precision = f"{optimal_epoch['precision_mean']*100:.2f}±{optimal_epoch['precision_std']*100:.2f}"
    recall = f"{optimal_epoch['recall_mean']*100:.2f}±{optimal_epoch['recall_std']*100:.2f}"
    f1 = f"{optimal_epoch['f1_mean']*100:.2f}±{optimal_epoch['f1_std']*100:.2f}"
    roc_auc = f"{optimal_epoch['roc_auc_mean']*100:.2f}±{optimal_epoch['roc_auc_std']*100:.2f}"
    pr_auc = f"{optimal_epoch['pr_auc_mean']*100:.2f}±{optimal_epoch['pr_auc_std']*100:.2f}"
    composite_score = f"{optimal_epoch['composite_score']:.4f}"
    

    print(f"{model}\t{accuracy}\t{precision}\t{recall}\t{f1}\t{roc_auc}\t{pr_auc}\t{composite_score}")


Model	Accuracy	Precision	Recall	F1	ROC AUC	PR AUC	Composite Score
DeepFE-PPI	94.54±0.65	96.81±0.48	92.13±0.92	94.41±0.70	94.54±0.66	93.13±0.86	0.9469
DeepTrio	91.77±0.56	95.50±0.57	87.69±1.51	91.42±0.68	91.77±0.56	89.90±0.53	0.9205
MyPPI	89.44±1.00	89.85±2.50	89.05±1.98	89.41±0.90	89.43±1.00	85.45±1.74	0.8960
PIPR	90.10±0.70	92.71±3.22	87.25±3.24	89.81±0.79	90.12±0.69	87.20±1.63	0.9035
TF-PPI	92.00±1.42	96.26±4.15	87.76±5.97	91.60±1.74	91.98±1.41	90.41±1.89	0.9218
